<a href="https://colab.research.google.com/github/ZhiYu-2002/copy-of-endonet-2022.1.4/blob/main/2022_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import io
import cv2
import h5py
import time
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from google.colab import output

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

class dataset_h5(Dataset):
    def __init__(self, h5file_path, annotation_file, transform):
        self.h5file = h5py.File(h5file_path, 'r')        
        self.annotations = pd.read_csv(annotation_file, index_col='Frame')
        self.transform = transform
        self.key = list(self.h5file.keys())
        
    def __len__(self):
        return len(self.key)

    def __getitem__(self, index):
        img_id = self.key[index]
        img = Image.open(io.BytesIO(np.array(self.h5file[img_id]))) # read the image from h5 file data
        y_label = torch.tensor(self.annotations.loc[img_id][0])

        if self.transform is not None:
            img = self.transform(img)
        return (img, y_label)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
t = time.time()

trainingLoaderSet = []
testingLoaderSet = []

for eachDataset in range(8):

  print("Processing dataset %d... " %(eachDataset), end='')
  # Load training dataset
  datasetPath = "/content/drive/Shareddrives/Surgery Video Research Team/Frames/cholec80/TrainingDataset_sub/trainingData"+str(eachDataset)+'.hdf5'
  labelPath = "/content/drive/Shareddrives/Surgery Video Research Team/Frames/cholec80/TrainingDataset_sub/trainingData"+str(eachDataset)+"_lables.txt"

  train_data = dataset_h5(h5file_path=datasetPath, annotation_file=labelPath, transform=transform)
  trainloader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=True, num_workers=1)

  trainingLoaderSet.append(trainloader)

  # Load testing dataset
  datasetPath = "/content/drive/Shareddrives/Surgery Video Research Team/Frames/cholec80/TestingDataset_sub/testingData"+str(eachDataset)+'.hdf5'
  labelPath = "/content/drive/Shareddrives/Surgery Video Research Team/Frames/cholec80/TestingDataset_sub/testingData"+str(eachDataset)+"_lables.txt"

  test_data = dataset_h5(h5file_path=datasetPath, annotation_file=labelPath, transform=transform)
  testloader = torch.utils.data.DataLoader(test_data, batch_size=50, shuffle=True, num_workers=1)

  testingLoaderSet.append(testloader)

  print("Finish!")
print("Load all the dataset costs %d minutes %d seconds"%((time.time()-t)/60, (time.time()-t)%60))

valLoaderSet = testingLoaderSet[0:2]

AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

AlexNet_model.eval()

AlexNet_model.classifier[6] = nn.Linear(4096,7)
AlexNet_model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

AlexNet_model.to(device)

param_classifier1 = []
param_classifier2 = []
for idx, m in enumerate(AlexNet_model.modules()):
  if idx == 18 or idx == 21:
    print(idx, '->', m)
    param_classifier1.append(m.weight)
  if idx == 23:
    print(idx, '->', m)
    param_classifier2.append(m.weight)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD([
      {'params': AlexNet_model.features.parameters()},
      {'params': param_classifier1},
      {'params': param_classifier2, 'lr': 0.01}], 
      lr=0.001, momentum=0.9)

optimizer_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=12, gamma=0.1)

AlexNet_model.train()
print("Is the model in training mode?",AlexNet_model.training)

def train_model(model, trainingLoaderSet, testingLoaderSet, criterion, optimizer, lr_scheduler, num_epochs = 30):
  '''
  It will return:
    1. Training and Testing loss record: running_Loss, val_Loss
    2. Training and Testing accuracy record: running_Acc, val_Acc
  '''
  running_Loss = []
  val_Loss = []
  running_Acc = []
  val_Acc = []

  startTime = time.time()

  startTime = time.time() # record the start time of training process
  pbar_all = tqdm(total=num_epochs, desc="Total training progress", ncols=700) # display the progress of training process

  for epoch in range(num_epochs):  # loop over the dataset multiple times

    loss_train = 0
    correct_train = 0
    total_train = 0
    loss_test = 0
    correct_test = 0
    total_test = 0

    model.train() # Trun the mode into training state

    # Training process
    pbar = tqdm(total=sum([len(item) for item in trainingLoaderSet]), desc="Epoch %d training progress"%(epoch), ncols=700) 
    for DatasetIndex, eachDataset in enumerate(trainingLoaderSet): # loop over the 8 sub-dataset
      for i, data in enumerate(eachDataset, 0):
        # send the input to GPU memory; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        
        optimizer.zero_grad()  # zero the parameter gradients  
        output = model(inputs) # forward propagation
        loss = criterion(output, labels) # caculate the loss between output and ground truth 
        loss.backward() # backward propagation        
        optimizer.step() # update the gradient to new gradients        
        loss_train += loss.item() # Caculate the loss and the number of correct predicted results 

        # caculate the running loss and accuracy
        model.eval() # turn the model into evaluation mode
        with torch.no_grad(): # turn off the gradient descent process
          output = model(inputs)
          _, predicted = torch.max(output.data, 1)
          correct_train += (predicted == labels).sum().item()
          total_train += labels.size(0)

        model.train()
        pbar.update(1)
    
    lr_scheduler.step()
    
    running_Loss.append([epoch, (loss_train / total_train)])
    running_Acc.append([epoch, (correct_train / total_train)])
    #pbar.close()


    

    # Testing process, evaluate the model through the testing dataset
    model.eval()
    with torch.no_grad():
      pbar = tqdm(total=sum([len(item) for item in testingLoaderSet]), desc="Epoch %d testing progress"%(epoch), ncols=700) 
      for DatasetIndex, eachDataset_test in enumerate(testingLoaderSet):
        for i, data in enumerate(eachDataset_test):
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          loss = criterion(outputs, labels) 
          _, predicted = torch.max(outputs.data, 1)       
          correct_test += (predicted == labels).sum().item()
          total_test += labels.size(0)
          loss_test += loss.item()

          pbar.update(1)

    val_Loss.append([epoch, (loss_test / total_test)])
    val_Acc.append([epoch, (correct_test / total_test)])
    #pbar.close()

    print('Epoch %d  |  Running Loss: %.6f  |  Validation Loss: %.6f  |  Running Accuracy: %.3f%%  |  Validataion Accuracy: %.3f%%  |  ' 
       %(epoch, (loss_train / total_train), (loss_test / total_test), (100 * correct_train / total_train), (100 * correct_test / total_test)))    
    print('Learning Rate:',end='')
    for item in optimizer.param_groups:
      print(' '+ str(item['lr']) + ',',end='')

    pbar_all.update(1)
  pbar_all.close()
  
  torch.save(model, '/content/drive/Shareddrives/Surgery Video Research Team/Code/EndoNetModel.pkl')
  print("Train the model cost total %d hours %d minutes"%((time.time() - startTime)/3600,(time.time() - startTime)%60))

  return running_Loss, val_Loss, running_Acc, val_Acc

running_Loss, val_Loss, running_Acc, val_Acc = train_model(AlexNet_model,
                                trainingLoaderSet, valLoaderSet, 
                                criterion, optimizer, lr_scheduler = optimizer_scheduler, 
                                num_epochs = 30)

print(running_Loss)
print(val_Loss) 
print(running_Acc) 
print(val_Acc)
plt.figure()
plt.plot([item[0] for item in running_Acc], [item[1] for item in running_Acc], label='Training Accuracy')
plt.plot([item[0] for item in val_Acc], [item[1] for item in val_Acc], label='Validation Accuracy')
for x in range(0,30,5):  
  y = running_Acc[x][1]
  s = str(round(running_Acc[x][1],2))
  plt.text(x-1, y, s, fontsize=10)
plt.text(29-1, running_Acc[29][1], round(running_Acc[29][1],2), fontsize=10)
for x in range(0,30,5):  
  y = val_Acc[x][1]
  s = str(round(val_Acc[x][1],2))
  plt.text(x-1, y, s, fontsize=10)
plt.text(29-1, val_Acc[29][1], round(val_Acc[29][1],2), fontsize=10)
plt.xlabel("# Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Epoch vs Accuracy")
plt.savefig('EpochvsAcc.jpg')
plt.show()

plt.figure()
plt.plot([item[0] for item in running_Loss], [item[1] for item in running_Loss], label = "Training Loss")
plt.plot([item[0] for item in val_Loss], [item[1] for item in val_Loss], label = "Validation Loss")
for x in range(0,30,5):  
  y = running_Loss[x][1]
  s = str(round(running_Loss[x][1],5))
  plt.text(x-1, y+0.0005, s, fontsize=10)
plt.text(29, running_Loss[29][1], round(running_Loss[29][1],5), fontsize=10)
for x in range(0,30,5):  
  y = val_Loss[x][1]
  s = str(round(val_Loss[x][1],4))
  plt.text(x-1, y+0.0005, s, fontsize=10)
plt.text(29, val_Loss[29][1], round(val_Loss[29][1],4), fontsize=10)

plt.xlabel("# Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Epoch vs Loss")
plt.savefig('EpochvsLoss.jpg')
plt.show()

AlexNet_model = torch.load("/content/drive/Shareddrives/Surgery Video Research Team/Code/EndoNetModel.pkl")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

AlexNet_model.to(device)

correct = 0
total = 0
predictedList = []
startTime = time.time()

AlexNet_model.eval()
pbar = tqdm(total=sum([len(item) for item in testingLoaderSet]), desc="Testing progress", ncols=700) 
with torch.no_grad():
  for DatasetIndex, eachDataset in enumerate(testingLoaderSet): 
    for i, data in enumerate(eachDataset):
      images, labels = data[0].to(device), data[1].to(device)
      outputs = AlexNet_model(images)
      _, predicted = torch.max(outputs.data, 1)
      predictedList.append([predicted.tolist(), labels.tolist(), DatasetIndex])

      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      pbar.update(1)
pbar.close()

print('Accuracy of the network on the testing dataset: %.2f %%' %(100 * correct / total))
print("Test the model cost total %d minutes"%((time.time() - startTime)/60))

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plotConfusionMatrix(y_true, y_predicted):
  confmat = confusion_matrix(y_true, y_predicted)
  plt.figure(figsize=(10,8))
  sn.heatmap(confmat, annot=True)
  plt.xlabel("Predicted phase")
  plt.ylabel("True phase")
  plt.show()

phaseTrue = []
phasePredicted = []

for index, item in enumerate(predictedList):
  phaseTrue += item[1]
  phasePredicted += item[0]

plotConfusionMatrix(phaseTrue, phasePredicted)

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score


print("Accuracy: %.2f%%"%(accuracy_score(phaseTrue, phasePredicted)*100))
print("Recll: %.2f%%"%(recall_score(phaseTrue, phasePredicted, average='weighted')*100))
print("F1 score: %.2f%%"%(f1_score(phaseTrue, phasePredicted, average='weighted')*100))
print("Precision score: %.2f%%"%(precision_score(phaseTrue, phasePredicted, average='weighted')*100))

Mounted at /content/drive
Processing dataset 0... 

OSError: ignored